### Compare CONUS2 and SNOTEL SWE ###
DTT, 06/23

This notebook reads in CSVs of CONUS2 daily mean flow [cms] and daily mean USGS flow [cms] and computes comparison statistics. Gages are matched between the two csv's to make sure that we're comparing viable gage locations. 

Inputs:
- CONUS2 daily mean flow csv in [cms] - the output of `Make_CONUS2_streamflow_csv.py`
- USGS daily mean flow csv in [cms] - the output of `point_obs_CONUS2_Streamflow.ipynb`
- USGS gage metadata csv - the output of `point_obs_CONUS2_Streamflow.ipynb`

Outputs:
- Gage summary CSV of statistics  
(bias, relative bias, correlation, Spearman rank, RMSE, NSE, scaled RMSE)
for CONUS 1.0 simulated streamflow, compared to USGS observations.

In [1]:
import sys
from parflow.tools.io import read_pfb,write_pfb
import numpy as np
import pandas as pd
#import hydroeval as he
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from math import sqrt
from scipy import stats

import matplotlib.pyplot as plt

In [5]:
water_year = 2003
# Directory containing output csv's from `1_Organize_Streamflow.py`
#organized_dir = '/glade/work/tijerina/PFCONUS2-analysis/scripts/Validation/Streamflow/Organized_Daily_Flow'
organized_dir = '/home/dtt2/CONUS2/PFCONUS2-analysis/scripts/Validation/WTD/Organized_Daily_WTD'

# names of csv files
pf_csv = 'PFCONUS2_Daily_matched_WTD_m_2003.csv'
obs_csv = 'USGS_Daily_matched_WTD_m_2003.csv'
metadata_csv = 'USGS_metadata_matched_WTD_m_2003.csv'

### Read CONUS2 & USGS flow

In [6]:
### read in CONUS2 daily flow matched csv
pfdata = pd.read_csv(f'{organized_dir}/{pf_csv}',index_col=['site_id'])
print(pfdata.shape)
pfdata.head()

(1004, 365)


,2002-10-01,2002-10-02,2002-10-03,2002-10-04,2002-10-05,2002-10-06,2002-10-07,2002-10-08,2002-10-09,2002-10-10,...,2003-09-21,2003-09-22,2003-09-23,2003-09-24,2003-09-25,2003-09-26,2003-09-27,2003-09-28,2003-09-29,2003-09-30
site_id,,,,,,,,,,,,,,,,,,,,,
290955098562101,0.046598,0.117977,0.220182,0.257405,0.284448,0.303802,0.317972,0.244333,0.002561,0.000000,...,0.066616,0.030094,0.036585,0.041375,0.045606,0.042465,0.046457,0.120872,0.222234,0.0
291136099375801,0.153622,0.322058,0.391806,0.436653,0.473755,0.503918,0.521985,0.256727,0.031601,0.020330,...,1.272928,1.244795,1.222374,1.213699,1.207622,1.197916,1.183117,1.171317,1.164986,0.0
292045099081801,0.179284,0.348520,0.415460,0.462341,0.501713,0.531404,0.538530,0.464910,0.064337,0.043442,...,1.252013,1.228831,1.211517,1.203631,1.199162,1.195820,1.195169,1.194164,1.195079,0.0
292117098524701,0.173391,0.401435,0.502473,0.568874,0.623283,0.662633,0.692886,0.459583,0.015222,0.037403,...,1.056265,0.976283,0.967152,0.980743,0.999757,1.010488,1.015186,1.029755,1.045833,0.0
292505098254002,0.032914,0.038503,0.043972,0.083968,0.311845,0.454438,0.513418,0.406310,0.001405,0.000000,...,1.143994,1.092434,1.073321,1.073828,1.084173,1.094581,1.101546,1.114014,1.128256,0.0


In [7]:
### read in USGS daily flow matched csv
usgsdata= pd.read_csv(f'{organized_dir}/{obs_csv}',index_col=['site_id']) 
print(usgsdata.shape)
usgsdata.head()

(1004, 365)


,2002-10-01,2002-10-02,2002-10-03,2002-10-04,2002-10-05,2002-10-06,2002-10-07,2002-10-08,2002-10-09,2002-10-10,...,2003-09-21,2003-09-22,2003-09-23,2003-09-24,2003-09-25,2003-09-26,2003-09-27,2003-09-28,2003-09-29,2003-09-30
site_id,,,,,,,,,,,,,,,,,,,,,
290955098562101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-14.825472,-14.819376,-14.828520,-14.871192,-14.892528,-14.935200,-14.938248,-14.907768,-14.892528,-14.901672
291136099375801,45.476160,49.371504,57.290208,55.22976,54.150768,52.776120,52.349400,49.685448,44.906184,43.120056,...,30.967680,30.540960,30.129480,29.708856,29.340048,28.959048,28.648152,28.382976,28.114752,27.852624
292045099081801,43.833288,43.988736,44.052744,44.08932,44.217336,44.189904,44.250864,44.171616,43.988736,43.665648,...,44.284392,44.162472,44.086272,43.958256,43.924728,43.897296,43.888152,43.918632,43.949112,43.943016
292117098524701,11.609832,11.667744,11.707368,11.78052,11.850624,11.868912,11.884152,11.853672,11.618976,11.478768,...,12.874752,12.780264,12.701016,12.594336,12.548616,12.515088,12.512040,12.539472,12.563856,12.621768
292505098254002,-19.808952,-19.699224,-19.607784,-19.56816,-19.641312,-19.598640,-19.559016,-19.644360,-19.891248,-20.083272,...,-20.241768,-20.116800,-20.238720,-20.369784,-20.321016,-20.543520,-20.531328,-20.616672,-20.659344,-20.622768


In [8]:
# change zero values in USGS df so that we can calculate stats without errors
usgsdata = usgsdata.where(usgsdata>0, 0.0001)

In [9]:
### read in USGS streamflow gage metadata for all gages with flow in October AND NWM csv for adjusted lat/long
#NWM_gage_csv = pd.read_csv('/glade/p/univ/ucsm0002/CONUS2/domain_files/NWM_Gage_Adjustments_final.csv')
usgs_gage_metadata = pd.read_csv(f'{organized_dir}/{metadata_csv}',index_col=['site_id'])
print(usgs_gage_metadata.shape)
usgs_gage_metadata.head()

(1004, 62)


,site_type,agency,site_name,site_id_agency,site_query_url,date_metadata_last_updated,latitude,longitude,state,huc,...,aqfr_type_cd,well_depth_va,hole_depth_va,depth_src_cd,project_no,conus1_x,conus1_y,conus2_x,conus2_y,num_obs
site_id,,,,,,,,,,,,,,,,,,,,,
290955098562101,groundwater well,USGS,TD-68-49-813 (Devine),290955098562101,https://waterservices.usgs.gov/nwis/site/?form...,2023-03-08,29.165778,-98.939411,TX,12110109,...,C,3200.0,3200.0,L,8653-00270,NaN,NaN,2019,485,327
291136099375801,groundwater well,USGS,YP-69-51-606 (E. Uvalde 3),291136099375801,https://waterservices.usgs.gov/nwis/site/?form...,2023-03-08,29.193573,-99.632830,TX,12110106,...,C,1400.0,1400.0,D,8653-BQ101,NaN,NaN,1951,490,338
292045099081801,groundwater well,USGS,TD-69-47-306,292045099081801,https://waterservices.usgs.gov/nwis/site/?form...,2023-03-08,29.346068,-99.138647,TX,12110107,...,C,1600.0,1600.0,S,8653-00270,NaN,NaN,2000,506,362
292117098524701,groundwater well,USGS,TD-68-41-301,292117098524701,https://waterservices.usgs.gov/nwis/site/?form...,2023-03-08,29.354957,-98.880028,TX,12100302,...,C,710.0,710.0,S,8653-00270,NaN,NaN,2025,506,331
292505098254002,groundwater well,USGS,AY-68-37-522 (San Antonio Transect A-2),292505098254002,https://waterservices.usgs.gov/nwis/site/?form...,2023-03-08,29.418289,-98.428071,TX,12100301,...,C,1075.0,1075.0,O,8653-00270,NaN,NaN,2069,512,363


### Calculate statistics

In [11]:
# set up DF for daily stats, index is gage site_id
stats_df = pd.DataFrame(index = usgs_gage_metadata.index, columns = ['latitude', 'longitude', 'conus2_x','conus2_y']) 
#, 'bias', 'pbias','absrelbias', 'relbias','srho', 'r2', 'mse', 'rmse', 'nse',scaled RMSE

stats_df['latitude'] = usgs_gage_metadata['latitude']
stats_df['longitude'] = usgs_gage_metadata['longitude']
stats_df['conus2_x'] = usgs_gage_metadata['conus2_x'].astype(int)
stats_df['conus2_y'] = usgs_gage_metadata['conus2_y'].astype(int)
stats_df['huc'] = usgs_gage_metadata['huc']
stats_df['HUC2']  = usgs_gage_metadata['huc'] // 1000000 #get the HUC2 while preserving the dtype=int64

print(stats_df.shape)
stats_df.head()

(1004, 6)


,latitude,longitude,conus2_x,conus2_y,huc,HUC2
site_id,,,,,,
290955098562101,29.165778,-98.939411,2019,485,12110109,12
291136099375801,29.193573,-99.632830,1951,490,12110106,12
292045099081801,29.346068,-99.138647,2000,506,12110107,12
292117098524701,29.354957,-98.880028,2025,506,12100302,12
292505098254002,29.418289,-98.428071,2069,512,12100301,12


In [12]:
# pfmean = np.mean(pfdata, axis = 1)
# pfmean

In [13]:
# compute annual average for observations
usgs_flow_sum = usgsdata.sum(axis=1)
stats_df['USGS_mean_mm'] = usgs_flow_sum/usgs_gage_metadata.num_obs
stats_df['PF_mean_mm'] = np.mean(pfdata, axis = 1)

In [14]:
# Set up array for stats
r2_daily = np.zeros(len(usgsdata.index))
srho_daily = np.zeros(len(usgsdata.index))
pval_daily = np.zeros(len(usgsdata.index))
mse_daily = np.zeros(len(usgsdata.index))
rmse_daily = np.zeros(len(usgsdata.index))
nse_daily = np.zeros(len(usgsdata.index))
bias_daily = np.zeros(len(usgsdata.index))
pbias_daily = np.zeros(len(usgsdata.index))
pbias_test = np.zeros(len(usgsdata.index))
absrelbias_daily = np.zeros(len(usgsdata.index))
relbias_daily = np.zeros(len(usgsdata.index))
r2_daily.shape
pf_minus_obs_daily = np.zeros(len(usgsdata.index))

In [15]:
for g in range(len(usgsdata.index)):
    gage = usgsdata.index[g]
    absrelbias_daily[g] = abs((pfdata.loc[gage].sum()-usgsdata.loc[gage].sum())/usgsdata.loc[gage].sum()) #absolute relative bias
    bias_daily[g] = (pfdata.loc[gage].sum()-usgsdata.loc[gage].sum())/usgsdata.loc[gage].sum()
    pbias_daily[g] = bias_daily[g]*100
    srho_daily[g], pval_daily[g] = stats.spearmanr(usgsdata.loc[gage], pfdata.loc[gage])
    mse_daily[g] = mean_squared_error(usgsdata.loc[gage], pfdata.loc[gage])
    rmse_daily[g] = sqrt(mean_squared_error(usgsdata.loc[gage], pfdata.loc[gage]))
    r2_daily[g] = r2_score(usgsdata.loc[gage], pfdata.loc[gage])
    pf_minus_obs_daily[g] = pfdata.loc[gage].sum()-usgsdata.loc[gage].sum()
   
    # #nse_daily[g] = he.nse(np.sum(pf_flow_matched.loc[gage]), np.sum(usgs_flow_matched.loc[gage]))
    #print(gage)
    # print(usgs_flow_matched.loc[gage])
    
    #import necessary libraries


#calculate RMSE
# sqrt(mean_squared_error(actual, pred)) 

/home/dtt2/.conda/envs/pf-mapping/lib/python3.9/site-packages/scipy/stats/_stats_py.py:4916: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(warn_msg))
/home/dtt2/.conda/envs/pf-mapping/lib/python3.9/site-packages/scipy/stats/_stats_py.py:4916: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(warn_msg))
/home/dtt2/.conda/envs/pf-mapping/lib/python3.9/site-packages/scipy/stats/_stats_py.py:4916: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(warn_msg))
/home/dtt2/.conda/envs/pf-mapping/lib/python3.9/site-packages/scipy/stats/_stats_py.py:4916: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(warn_msg))
/home/dtt2/.conda/envs/pf-mapping/li

In [16]:
# assign arrays to pandas columns in stats_df
stats_df['pf_minus_obs'] = pf_minus_obs_daily
stats_df['absrelbias'] = absrelbias_daily
stats_df['bias'] = bias_daily
stats_df['pbias'] = pbias_daily
stats_df['srho'] = srho_daily
stats_df['r2'] = r2_daily
stats_df['rmse'] = rmse_daily
stats_df['mse'] = mse_daily




In [17]:
stats_df.head()

,latitude,longitude,conus2_x,conus2_y,huc,HUC2,USGS_mean_mm,PF_mean_mm,pf_minus_obs,absrelbias,bias,pbias,srho,r2,rmse,mse
site_id,,,,,,,,,,,,,,,,
290955098562101,29.165778,-98.939411,2019,485,12110109,12,0.000112,0.246099,89.789752,2459.993198,2459.993198,245999.319760,NaN,-5.301381e+38,0.312043,0.097371
291136099375801,29.193573,-99.632830,1951,490,12110106,12,30.575497,0.795623,-10044.115448,0.971900,-0.971900,-97.189975,0.555330,-5.608879e+00,29.799566,888.014144
292045099081801,29.346068,-99.138647,2000,506,12110107,12,43.010270,0.787037,-15282.449350,0.981550,-0.981550,-98.154955,0.750594,-6.788512e+01,42.160595,1777.515773
292117098524701,29.354957,-98.880028,2025,506,12100302,12,11.646980,0.730964,-3588.348393,0.930793,-0.930793,-93.079337,0.801893,-4.884833e+00,10.638994,113.188199
292505098254002,29.418289,-98.428071,2069,512,12100301,12,0.000101,0.577122,210.613008,5770.219389,5770.219389,577021.938861,NaN,-3.635421e+39,0.817142,0.667721


In [18]:
stats_df.to_csv(f'{organized_dir}/Summary_WTD_Daily_CONUS2_WY2003.csv')